In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import os
import json
from dotenv import load_dotenv
from google import genai
from pydantic import BaseModel, Field
from typing import List

load_dotenv()

SPREADSHEET_ID = '1Au5SOOeJRktvDNoFYGeUPsvRfS5GhhG3ULUTwxQX-Sc'
SHEET_NAME = 'interview_tracker'
TRANSCRIPT_COLUMN_INDEX = 8 
def get_google_sheets_service():
    try:
        SERVICE_ACCOUNT_FILE = os.path.join(os.getcwd(), "credentials.json")
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=["https://www.googleapis.com/auth/spreadsheets"]
        )
        return build("sheets", "v4", credentials=credentials)
    except Exception as e:
        print(f"Failed to fetch sheets service: {e}")

def fetch_transcript_by_code(code: str):
    service = get_google_sheets_service()
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=f"{SHEET_NAME}!A:I"
    ).execute()

    rows = result.get("values", [])
    for row in rows:
        if row and row[0] == code:
            return row[TRANSCRIPT_COLUMN_INDEX] if len(row) > TRANSCRIPT_COLUMN_INDEX else None
    return None

class QuestionAssessment(BaseModel):
    question: str
    candidate_response: str
    answer_correctness: str
    feedback: str

class TopicEvaluation(BaseModel):
    topic: str
    assessments: List[QuestionAssessment]

class InterviewEvaluation(BaseModel):
    topics: List[TopicEvaluation]

api_key = os.getenv('GOOGLE_API_KEY')

def evaluate_transcript(transcript):
    evaluation_prompt = f"""
    # Technical Interview Evaluation Task
    ## Interview Transcript
    {transcript}
    ## Evaluation Instructions:
    Analyze this technical interview transcript and provide a structured evaluation of the candidate's performance.
    1. Carefully review the candidate's responses based on the questions in the transcript.
    2. For each question, determine if it was answered correctly (Yes/No).
    3. Assess:
    - Technical accuracy and understanding.
    4. Generate a structured report categorized by topics
    """

    client = genai.Client(api_key=api_key)
    model_response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=evaluation_prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': InterviewEvaluation,
        }
    )
    return model_response

def write_evaluation_to_new_sheet(code, dataframe: pd.DataFrame):
    service = get_google_sheets_service()
    sheet = service.spreadsheets()

    # Create a new sheet
    requests = [{
        "addSheet": {
            "properties": {
                "title": code
            }
        }
    }]
    sheet.batchUpdate(spreadsheetId=SPREADSHEET_ID, body={"requests": requests}).execute()

    values = [list(dataframe.columns)] + dataframe.values.tolist()

    # Write to the new sheet
    sheet.values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=f"{code}!A1",
        valueInputOption="RAW",
        body={"values": values}
    ).execute()


def run_evaluation_for_code(code: str):
    transcript = fetch_transcript_by_code(code)
    if not transcript:
        print(f"No transcript found for code: {code}")
        return

    response = evaluate_transcript(transcript)
    evaluation = json.loads(response.text)

    data = []
    for topic in evaluation['topics']:
        for assessment in topic['assessments']:
            data.append({
                "Topic": topic['topic'],
                "Question": assessment['question'],
                "Candidate Response": assessment['candidate_response'],
                "Answer Correctness": assessment['answer_correctness'],
                "Feedback": assessment['feedback']
            })

    df = pd.DataFrame(data)
    write_evaluation_to_new_sheet(code, df)
    print(f"Evaluation written to sheet: {code}")

# Example usage:
run_evaluation_for_code("291574130324")


Evaluation written to sheet: 291574130324
